In [14]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from network import VG_nn

In [11]:
df = pd.read_csv("training.csv")
features = ["sigma", "theta", "kappa", "strike", "T"]
param_range = {param: [min(df[param]), max(df[param])] for param in df.columns}
df.head()

,sigma,theta,kappa,strike,T,expected_payoff,error,Ntraj
0,0.11,-0.34,0.11,1.00,0.25,0.030152,0.002309,1000
1,0.11,-0.34,0.11,0.95,0.25,0.064010,0.003272,1000
2,0.11,-0.34,0.11,0.90,0.25,0.104068,0.003996,1000
3,0.11,-0.34,0.11,0.85,0.25,0.153755,0.004293,1000
4,0.11,-0.34,0.12,1.00,0.25,0.030387,0.002263,1000


In [12]:
X_train = torch.tensor(df[features].values, dtype=torch.float32)
y_train = torch.tensor(df["expected_payoff"], dtype=torch.float32)
X_train.shape, X_train, y_train.shape, y_train

(torch.Size([16000, 5]),
 tensor([[ 0.1100, -0.3400,  0.1100,  1.0000,  0.2500],
         [ 0.1100, -0.3400,  0.1100,  0.9500,  0.2500],
         [ 0.1100, -0.3400,  0.1100,  0.9000,  0.2500],
         ...,
         [ 0.2000, -0.0600,  0.2000,  0.9500,  2.0000],
         [ 0.2000, -0.0600,  0.2000,  0.9000,  2.0000],
         [ 0.2000, -0.0600,  0.2000,  0.8500,  2.0000]]),
 torch.Size([16000]),
 tensor([0.0302, 0.0640, 0.1041,  ..., 0.1455, 0.1592, 0.1984]))

In [34]:
dataset = TensorDataset(X_train, y_train)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

vg_nn = VG_nn()
optimizer = torch.optim.AdamW(vg_nn.parameters(), lr=0.1)

In [35]:
n_epochs = 20

for e in range(n_epochs):
    lossi = []
    for X_batch, y_batch in dataloader:
        y_out = vg_nn(X_batch)
        loss = torch.mean((y_out - y_batch)**2)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lossi.append(loss.item())
        # for name, param in vg_nn.named_parameters():
        #     if name == "net.0.weight":
        #         print(f"Parameter name: {name}")
        #         print(f"Shape: {param.shape}")
        #         print(f"Values: {param.data[0]}\n")
    print(f"Mean loss on the {e}-th epoch: {np.array(lossi).mean()}")

Mean loss on the 0-th epoch: 0.7568654671208933
Mean loss on the 1-th epoch: 0.01431115161627531
Mean loss on the 2-th epoch: 0.014311151541769505
Mean loss on the 3-th epoch: 0.014311151597648859
Mean loss on the 4-th epoch: 0.014311151545494795
Mean loss on the 5-th epoch: 0.014311151567846537
Mean loss on the 6-th epoch: 0.014311151687055827
Mean loss on the 7-th epoch: 0.014311151575297117
Mean loss on the 8-th epoch: 0.014311151653528214
Mean loss on the 9-th epoch: 0.014311151571571827
Mean loss on the 10-th epoch: 0.014311151560395956
Mean loss on the 11-th epoch: 0.01431115159392357
Mean loss on the 12-th epoch: 0.014311151623725892
Mean loss on the 13-th epoch: 0.014311151515692472
Mean loss on the 14-th epoch: 0.014311151575297117
Mean loss on the 15-th epoch: 0.014311151579022408
Mean loss on the 16-th epoch: 0.014311151545494795
Mean loss on the 17-th epoch: 0.014311151508241892
Mean loss on the 18-th epoch: 0.014311151634901762
Mean loss on the 19-th epoch: 0.0143111515715